# 🧪 Project Chimera 2.0 — Phase 1: The Data Kitchen
---
**Objective:** Ingest massive, heterogeneous raw datasets (Tweets, 1-Min Crypto, Daily Macro) and fuse them into a single, clean, hourly-aligned Parquet file ready for feature engineering.

**Scientific Alpha Logic:**
> Markets are driven by a confluence of signals operating at different frequencies.
> By aligning sentiment (social), price action (market microstructure), and macro-economic regime indicators onto a single hourly timeline, we create a unified information surface that no single data source can provide alone.

**Output:** `chimera_master_dataset.parquet`

## Step 0 — Environment Setup & Configuration

In [2]:
# ============================================================================
# STEP 0: ENVIRONMENT SETUP & CONFIGURATION
# ============================================================================
# Install / verify dependencies (Kaggle kernels have most pre-installed)
import subprocess, sys

def _ensure_package(pkg_name, import_name=None):
    """Silently install a package if not already available."""
    import_name = import_name or pkg_name
    try:
        __import__(import_name)
    except ImportError:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", pkg_name]
        )

_ensure_package("nltk")
_ensure_package("tqdm")
_ensure_package("pyarrow")          # Parquet backend

# --- Core Imports -----------------------------------------------------------
import os, glob, re, warnings, gc, time
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import nltk
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 60)
pd.set_option("display.float_format", "{:.4f}".format)

# --- NLTK VADER Lexicon (download once) --------------------------------------
nltk.download("vader_lexicon", quiet=True)
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# --- Paths -------------------------------------------------------------------
# Kaggle mounts datasets in one of three layouts:
#   A) /kaggle/input/<dataset-slug>/                        (classic)
#   B) /kaggle/input/datasets/<dataset-slug>/               (newer)
#   C) /kaggle/input/datasets/<author>/<dataset-slug>/      (API / newest)
# We build a flat list of ALL leaf dataset directories regardless of nesting,
# then search that list by keyword.

_base = Path("/kaggle/input")
OUTPUT_DIR  = Path("/kaggle/working")
OUTPUT_FILE = OUTPUT_DIR / "chimera_master_dataset.parquet"

# ---- Collect every dataset directory (leaf dirs that contain files) ----------
def _collect_dataset_dirs(root: Path, max_depth: int = 4) -> list[Path]:
    """Walk up to *max_depth* levels and return dirs that contain files."""
    results = []
    def _walk(p: Path, depth: int):
        if depth > max_depth:
            return
        try:
            children = sorted(p.iterdir())
        except PermissionError:
            return
        has_files = any(c.is_file() for c in children)
        if has_files:
            results.append(p)
        for c in children:
            if c.is_dir():
                _walk(c, depth + 1)
    _walk(root, 0)
    return results

ALL_DATASET_DIRS = _collect_dataset_dirs(_base)

print(f"📂 Found {len(ALL_DATASET_DIRS)} dataset folder(s) under {_base}:")
for d in ALL_DATASET_DIRS:
    print(f"   • {d.relative_to(_base)}")

# ---- Helper: find the main CSV inside a directory ---------------------------
def find_csv(directory: Path, hint: str = "") -> Path:
    """Robustly locate the primary CSV in *directory* (recursive)."""
    candidates = sorted(directory.rglob("*.csv"))
    if not candidates:
        raise FileNotFoundError(f"No CSV found in {directory}")
    if hint:
        matched = [c for c in candidates if hint.lower() in c.name.lower()]
        if matched:
            return matched[0]
    return max(candidates, key=lambda p: p.stat().st_size)

# ---- Auto-discover dataset directories by keyword ---------------------------
def _find_dataset_dir(keyword: str, alt_keywords: list[str] = None) -> Path:
    """Search ALL_DATASET_DIRS for the first dir whose full path matches."""
    all_kw = [keyword] + (alt_keywords or [])
    for kw in all_kw:
        for d in ALL_DATASET_DIRS:
            if kw.lower() in str(d).lower():
                return d
    raise FileNotFoundError(
        f"Cannot find dataset directory matching any of {all_kw}.\n"
        f"Available: {[str(d.relative_to(_base)) for d in ALL_DATASET_DIRS]}"
    )

# ---- Dataset paths (resolved dynamically) -----------------------------------
DIR_BTC        = _find_dataset_dir("bitcoin-historical", ["btc-usd", "binance"])
DIR_TWEETS     = _find_dataset_dir("bitcoin-tweet",      ["tweet", "twitter"])
DIR_SPX        = _find_dataset_dir("gspc",               ["sp500", "s-p-500", "s-and-p"])
DIR_DXY        = _find_dataset_dir("trade-weighted",     ["dollar-index", "dtwex", "dxy"])
DIR_FEAR_GREED = _find_dataset_dir("fear",               ["greed", "fgi"])
DIR_MACRO      = _find_dataset_dir("macro",              ["vital-signs", "economic"])

# Resolve exact file paths
PATH_BTC_1M     = find_csv(DIR_BTC,        hint="1m")
PATH_TWEETS     = find_csv(DIR_TWEETS,     hint="tweet")
PATH_SPX_DIR    = DIR_SPX
PATH_DXY_DIR    = DIR_DXY
PATH_FEAR_GREED = find_csv(DIR_FEAR_GREED, hint="fear")
PATH_MACRO_DIR  = DIR_MACRO

# Show what we resolved
print(f"\n   BTC 1-Min   : {PATH_BTC_1M}")
print(f"   Tweets      : {PATH_TWEETS}")
print(f"   S&P 500 dir : {PATH_SPX_DIR}")
print(f"   DXY dir     : {PATH_DXY_DIR}")
print(f"   Fear & Greed: {PATH_FEAR_GREED}")
print(f"   Macro dir   : {PATH_MACRO_DIR}")

print(f"\n✅ Environment ready.")
print(f"   NumPy  {np.__version__}  •  Pandas {pd.__version__}")
print(f"   Output → {OUTPUT_FILE}")

📂 Found 6 dataset folder(s) under /kaggle/input:
   • datasets/eswaranmuthu/u-s-economic-vital-signs-25-years-of-macro-data
   • datasets/kaushiksuresh147/bitcoin-tweets
   • datasets/liiucbs/crypto-fear-and-greed-index
   • datasets/novandraanugrah/bitcoin-historical-datasets-2018-2024
   • datasets/organizations/federalreserve/real-trade-weighted-u.s.-dollar-index-collection
   • datasets/rezanematpour/historical-s-and-p-500-gspc-index-data-19272025

   BTC 1-Min   : /kaggle/input/datasets/novandraanugrah/bitcoin-historical-datasets-2018-2024/btc_15m_data_2018_to_2025.csv
   Tweets      : /kaggle/input/datasets/kaushiksuresh147/bitcoin-tweets/Bitcoin_tweets.csv
   S&P 500 dir : /kaggle/input/datasets/rezanematpour/historical-s-and-p-500-gspc-index-data-19272025
   DXY dir     : /kaggle/input/datasets/organizations/federalreserve/real-trade-weighted-u.s.-dollar-index-collection
   Fear & Greed: /kaggle/input/datasets/liiucbs/crypto-fear-and-greed-index/CryptoGreedFear.csv
   Macro dir

## Step 1 — Sentiment Extraction (Memory-Safe Chunked Pipeline)

**Why chunked?** The Bitcoin Tweets dataset contains millions of rows. Loading it in one shot will exceed Kaggle's RAM ceiling (≈30 GB).

**Alpha Logic:** Social-media sentiment is a *leading* indicator of retail positioning. By aggregating VADER compound scores to the hourly grain, we capture the crowd's emotional state right before price moves — giving the model a behavioural edge that pure price data cannot.

In [3]:
# ============================================================================
# STEP 1: SENTIMENT EXTRACTION — MEMORY-SAFE CHUNKED PIPELINE
# ============================================================================
# Scientific Alpha:
#   VADER (Valence Aware Dictionary and sEntiment Reasoner) is specifically
#   tuned for social-media text.  It handles slang, emojis, and punctuation
#   emphasis (e.g., "AMAZING!!!") out of the box — critical for crypto Twitter.
# ============================================================================

CHUNK_SIZE = 100_000   # rows per chunk — keeps peak RAM well under 4 GB

sia = SentimentIntensityAnalyzer()

# --- Crypto-specific lexicon update ------------------------------------------
# VADER's default lexicon was trained on general social-media text and misses
# crypto-native slang entirely.  Without this update, terms like "moon",
# "rekt", or "hodl" receive near-zero scores, flattening the sentiment signal.
# These manually-curated scores align with empirical crypto-Twitter semantics.
crypto_lexicon = {
    # Positive / bullish terms  (scores pushed to extremes for sharper signal)
    'moon': 5.0, 'bullish': 3.5, 'hodl': 2.0, 'ath': 4.0, 'btfd': 2.5,
    # Negative / bearish terms
    'rekt': -5.0, 'bearish': -3.5, 'rug': -5.0, 'fud': -3.0, 'dump': -3.0,
}
sia.lexicon.update(crypto_lexicon)
print(f"🧠 VADER lexicon updated with {len(crypto_lexicon)} crypto-specific terms")

# --- Auto-detect timestamp column (used by Steps 3A–3D for macro data) -------
def _detect_ts_col(cols):
    """Pick the best timestamp column from a list of column names."""
    for candidate in ["timestamp", "date", "created_at", "datetime", "Timestamp", "Date"]:
        if candidate in cols:
            return candidate
    for c in cols:
        if "time" in c.lower() or "date" in c.lower():
            return c
    raise KeyError(f"Cannot auto-detect timestamp column from {list(cols)}")

# --- Text cleaning regex (compiled once for speed) ---------------------------
RE_URL      = re.compile(r"https?://\S+|www\.\S+")
RE_USERNAME = re.compile(r"@\w+")
RE_HASHTAG  = re.compile(r"#")           # Remove the '#' symbol but KEEP the word
RE_MULTI_WS = re.compile(r"\s+")

def clean_tweet(text: str) -> str:
    """Strip URLs, @mentions, '#' symbols.  KEEP emojis (VADER scores them)."""
    if not isinstance(text, str):
        return ""
    text = RE_URL.sub("", text)
    text = RE_USERNAME.sub("", text)
    text = RE_HASHTAG.sub("", text)       # '#Bitcoin' → 'Bitcoin'
    text = RE_MULTI_WS.sub(" ", text).strip()
    return text

def vader_compound(text: str) -> float:
    """Return VADER compound score (–1 … +1)."""
    return sia.polarity_scores(text)["compound"]

# --- Chunked processing loop ------------------------------------------------
hourly_chunks: list[pd.DataFrame] = []
total_tweets = 0

print(f"📖 Reading tweets in chunks of {CHUNK_SIZE:,} from:\n   {PATH_TWEETS}\n")

t0 = time.time()

reader = pd.read_csv(PATH_TWEETS, chunksize=CHUNK_SIZE, lineterminator="\n",
                      on_bad_lines="skip", engine="c")

for i, chunk in enumerate(tqdm(reader, desc="Sentiment chunks", unit="chunk")):

    # ── 1) Robust Date Parsing (FIX for all-zero sentiment bug) ─────────
    try:
        # 1a. Explicitly target the 'date' column (format: "YYYY-MM-DD HH:MM:SS")
        chunk['timestamp'] = pd.to_datetime(chunk['date'], errors='coerce')

        # 1b. Strip timezone if present (safety net — Binance data is tz-naive)
        if pd.api.types.is_datetime64_any_dtype(chunk['timestamp']):
            if chunk['timestamp'].dt.tz is not None:
                chunk['timestamp'] = chunk['timestamp'].dt.tz_localize(None)

        # 1c. Floor to nearest hour (critical for merging with hourly market data)
        chunk['hour'] = chunk['timestamp'].dt.floor('h')

        # 1d. Drop rows where date parsing failed
        chunk.dropna(subset=['hour'], inplace=True)

    except Exception as e:
        print(f"⚠️ Error parsing date in chunk {i}: {e}")
        continue

    # 2) Detect the text column
    text_col = None
    for candidate in ["text", "tweet", "content", "Tweet", "Text"]:
        if candidate in chunk.columns:
            text_col = candidate
            break
    if text_col is None:
        # Fallback: longest-average-string column (heuristic)
        str_cols = chunk.select_dtypes(include="object").columns
        text_col = max(str_cols, key=lambda c: chunk[c].astype(str).str.len().mean())

    # 3) Clean & score
    chunk["clean_text"]      = chunk[text_col].apply(clean_tweet)
    chunk["vader_compound"]  = chunk["clean_text"].apply(vader_compound)

    # 4) Aggregate to hourly grain IMMEDIATELY (memory-safe)
    agg = (
        chunk.groupby("hour")
        .agg(
            sentiment_mean=("vader_compound", "mean"),
            sentiment_std =("vader_compound", "std"),
            tweet_volume  =("vader_compound", "size"),
        )
    )
    hourly_chunks.append(agg)
    total_tweets += len(chunk)

    # Free memory
    del chunk, agg
    gc.collect()

elapsed = time.time() - t0

# --- Combine all hourly aggregations ----------------------------------------
# Because the same hour can span multiple chunks, we do a weighted re-agg.
df_sentiment_raw = pd.concat(hourly_chunks)
del hourly_chunks
gc.collect()

# Weighted mean sentiment across chunks for the same hour
df_sentiment_hourly = (
    df_sentiment_raw
    .groupby(level=0)
    .apply(
        lambda g: pd.Series({
            "sentiment_mean": np.average(g["sentiment_mean"], weights=g["tweet_volume"]),
            "sentiment_std" : g["sentiment_std"].mean(),       # approx pooled std
            "tweet_volume"  : g["tweet_volume"].sum(),
        })
    )
)
df_sentiment_hourly.index.name = "timestamp"

del df_sentiment_raw
gc.collect()

print(f"\n✅ Sentiment done — {total_tweets:,} tweets → "
      f"{len(df_sentiment_hourly):,} hourly rows  ({elapsed:.0f}s)")
df_sentiment_hourly.head()

🧠 VADER lexicon updated with 10 crypto-specific terms
📖 Reading tweets in chunks of 100,000 from:
   /kaggle/input/datasets/kaushiksuresh147/bitcoin-tweets/Bitcoin_tweets.csv



Sentiment chunks: 0chunk [00:00, ?chunk/s]


✅ Sentiment done — 4,689,288 tweets → 4,039 hourly rows  (1295s)


,sentiment_mean,sentiment_std,tweet_volume
timestamp,,,
2021-02-05 10:00:00,0.3789,0.3420,11.0000
2021-02-05 11:00:00,0.1088,0.3642,88.0000
2021-02-05 12:00:00,0.2038,0.3425,139.0000
2021-02-05 13:00:00,0.1314,0.3307,131.0000
2021-02-05 14:00:00,0.1664,0.3388,160.0000


## Step 2 — Market Data Processing (The Backbone)

**Alpha Logic:** 1-minute Binance candles are the highest-fidelity price signal available. Resampling to 1-Hour OHLCV produces the **master timeline** — every other signal gets aligned to this index. The market dataframe is the "spine" of the final dataset; its index defines which hours exist.

In [4]:
# ============================================================================
# STEP 2: MARKET DATA PROCESSING — THE BACKBONE
# ============================================================================
# Scientific Alpha:
#   The Binance BTC/USD dataset captures micro-structure moves.  By
#   resampling to 1H OHLCV we preserve intra-hour range (High-Low)
#   and volume dynamics — both proven alpha signals in momentum literature.
# ============================================================================

print(f"📖 Loading BTC candles from:\n   {PATH_BTC_1M}\n")

df_btc_raw = pd.read_csv(PATH_BTC_1M)

print(f"   Columns found: {list(df_btc_raw.columns)}")
print(f"   Raw rows: {len(df_btc_raw):,}")
print(f"   First row sample:\n{df_btc_raw.head(2).to_string()}\n")

# --- Auto-detect columns ----------------------------------------------------
col_map = {}
for col in df_btc_raw.columns:
    cl = col.strip().lower()
    if cl in ("timestamp", "date", "datetime", "time", "open_time", "open time"):
        col_map["timestamp"] = col
    elif cl == "open":
        col_map["open"] = col
    elif cl == "high":
        col_map["high"] = col
    elif cl == "low":
        col_map["low"] = col
    elif cl == "close":
        col_map["close"] = col
    elif cl in ("volume", "vol", "volume_(btc)", "volume btc"):
        col_map["volume"] = col

# If no timestamp column found, check if the index is datetime-like, or
# if there's an unnamed first column that looks like dates
if "timestamp" not in col_map:
    # Check for 'Unnamed: 0' or first column being a date
    first_col = df_btc_raw.columns[0]
    sample_val = str(df_btc_raw[first_col].dropna().iloc[0])
    # Test if it parses as a date
    try:
        pd.to_datetime(sample_val)
        col_map["timestamp"] = first_col
        print(f"   → Auto-detected '{first_col}' as timestamp column")
    except (ValueError, TypeError):
        # Last resort: try to parse the index
        try:
            pd.to_datetime(df_btc_raw.index[:5])
            df_btc_raw = df_btc_raw.reset_index()
            col_map["timestamp"] = df_btc_raw.columns[0]
            print(f"   → Using index as timestamp column")
        except (ValueError, TypeError):
            raise KeyError(
                f"Cannot find timestamp column. Columns: {list(df_btc_raw.columns)}\n"
                f"First row: {df_btc_raw.iloc[0].to_dict()}"
            )

print(f"   Detected mapping: {col_map}")

# --- Parse & set datetime index ----------------------------------------------
ts_col = col_map["timestamp"]

sample = df_btc_raw[ts_col].dropna().iloc[0]
if isinstance(sample, (int, float, np.integer, np.floating)) or str(sample).isdigit():
    val = int(float(str(sample)))
    unit = "ms" if val > 1e12 else "s"
    df_btc_raw["timestamp"] = pd.to_datetime(df_btc_raw[ts_col].astype(float), unit=unit, utc=True)
else:
    df_btc_raw["timestamp"] = pd.to_datetime(df_btc_raw[ts_col], utc=True, errors="coerce")

df_btc_raw.set_index("timestamp", inplace=True)
df_btc_raw.sort_index(inplace=True)

# --- Rename to canonical names -----------------------------------------------
rename = {col_map[k]: k for k in ("open", "high", "low", "close", "volume") if k in col_map}
df_btc_raw.rename(columns=rename, inplace=True)

# --- Resample to 1-Hour OHLCV -----------------------------------------------
df_market = df_btc_raw[["open", "high", "low", "close", "volume"]].resample("1h").agg({
    "open":   "first",
    "high":   "max",
    "low":    "min",
    "close":  "last",
    "volume": "sum",
}).dropna()

# Make index timezone-naive for uniform merging later
if df_market.index.tz is not None:
    df_market.index = df_market.index.tz_localize(None)
df_market.index.name = "timestamp"

del df_btc_raw
gc.collect()

print(f"\n✅ Market backbone ready — {len(df_market):,} hourly candles")
print(f"   Range: {df_market.index.min()} → {df_market.index.max()}")
df_market.head()

📖 Loading BTC candles from:
   /kaggle/input/datasets/novandraanugrah/bitcoin-historical-datasets-2018-2024/btc_15m_data_2018_to_2025.csv

   Columns found: ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
   Raw rows: 284,589
   First row sample:
                     Open time       Open       High        Low      Close   Volume                   Close time  Quote asset volume  Number of trades  Taker buy base asset volume  Taker buy quote asset volume  Ignore
0  2018-01-01 00:00:00.000000  13715.6500 13715.6500 13400.0100 13556.1500 123.6160  2018-01-01 00:14:59.999000         1675544.8866              1572                      63.2271                   857610.8270       0
1  2018-01-01 00:15:00.000000  13533.7500 13550.8700 13402.0000 13521.1200  98.1364  2018-01-01 00:29:59.999000         1321756.8518              1461                      47.6864 

,open,high,low,close,volume
timestamp,,,,,
2018-01-01 00:00:00,13715.6500,13715.6500,13400.0100,13529.0100,443.3562
2018-01-01 01:00:00,13528.9900,13595.8900,13155.3800,13203.0600,383.6970
2018-01-01 02:00:00,13203.0000,13418.4300,13200.0000,13330.1800,429.0646
2018-01-01 03:00:00,13330.2600,13611.2700,13290.0000,13410.0300,420.0870
2018-01-01 04:00:00,13434.9800,13623.2900,13322.1500,13601.0100,340.8073


## Step 3 — Macro & Auxiliary Data Fusion

**Alpha Logic:** Cross-asset correlation regimes shift over time. Including S&P 500, DXY, Fear & Greed, and CPI/Interest-Rate data lets the model learn *when* BTC behaves as a risk-on proxy vs. a dollar-hedge vs. an uncorrelated asset — a structural edge for regime detection.

**Forward-Fill Rule:** Daily/monthly data is forward-filled to hourly so that every row reflects *only information available at that point in time*. NO backward fill — that would leak future data.

In [5]:
# ============================================================================
# STEP 3A: S&P 500 (DAILY → HOURLY)
# ============================================================================
# Scientific Alpha:
#   S&P 500 is the global risk barometer.  BTC's correlation with equities
#   has increased post-2020 as institutional allocators treat it as a risk
#   asset.  Including SPX lets the model capture risk-on / risk-off regime
#   shifts that precede crypto moves.
# ============================================================================

print("📖 Loading S&P 500 data ...")

try:
    spx_path = find_csv(PATH_SPX_DIR, hint="sp500")
except FileNotFoundError:
    spx_path = find_csv(PATH_SPX_DIR)  # fallback: largest CSV

print(f"   Found: {spx_path.name}")

df_spx_raw = pd.read_csv(spx_path)

# Auto-detect date & close columns
spx_date_col  = _detect_ts_col(df_spx_raw.columns)
spx_close_col = None
for c in df_spx_raw.columns:
    if "close" in c.lower() or "adj" in c.lower() or "price" in c.lower():
        spx_close_col = c
        break
if spx_close_col is None:
    num_cols = df_spx_raw.select_dtypes(include="number").columns
    spx_close_col = [c for c in num_cols if "vol" not in c.lower()][0]

df_spx = pd.DataFrame({
    "timestamp": pd.to_datetime(df_spx_raw[spx_date_col], errors="coerce"),
    "spx_close": pd.to_numeric(df_spx_raw[spx_close_col], errors="coerce"),
}).dropna()

df_spx.set_index("timestamp", inplace=True)
df_spx.sort_index(inplace=True)

if df_spx.index.tz is not None:
    df_spx.index = df_spx.index.tz_localize(None)

# Resample Daily → Hourly with forward-fill (NO backward fill = no leakage)
df_spx_hourly = df_spx.resample("1h").ffill()

print(f"✅ S&P 500 — {len(df_spx):,} daily → {len(df_spx_hourly):,} hourly rows")
del df_spx_raw, df_spx
gc.collect()

📖 Loading S&P 500 data ...
   Found: SP500.csv
✅ S&P 500 — 24,532 daily → 856,129 hourly rows


0

In [6]:
# ============================================================================
# STEP 3B: DXY — U.S. DOLLAR INDEX (DAILY → HOURLY)
# ============================================================================
# Scientific Alpha:
#   BTC is priced in USD.  A strengthening dollar (rising DXY) typically
#   pressures all risk assets.  The DXY captures global liquidity tightening
#   signals earlier than crypto-native indicators.
# ============================================================================

print("📖 Loading DXY Dollar Index ...")

# The dataset contains multiple FRED series; pick DTWEXB (Broad) as the
# single best proxy for overall dollar strength.
try:
    dxy_path = find_csv(PATH_DXY_DIR, hint="dtwexb")
except FileNotFoundError:
    try:
        dxy_path = find_csv(PATH_DXY_DIR, hint="dollar")
    except FileNotFoundError:
        dxy_path = find_csv(PATH_DXY_DIR)

print(f"   Found: {dxy_path.name}")

df_dxy_raw = pd.read_csv(dxy_path)

dxy_date_col = _detect_ts_col(df_dxy_raw.columns)

# Find the value column (could be 'DTWEXBGS', 'Value', 'Close', etc.)
dxy_val_col = None
for c in df_dxy_raw.columns:
    cl = c.strip().lower()
    if cl in ("value", "close", "price", "dtwexbgs", "dtwexb", "dtwexm", "index"):
        dxy_val_col = c
        break
if dxy_val_col is None:
    num_cols = df_dxy_raw.select_dtypes(include="number").columns.tolist()
    num_cols = [c for c in num_cols if "year" not in c.lower() and "id" not in c.lower()]
    if num_cols:
        dxy_val_col = num_cols[0]
    else:
        non_date = [c for c in df_dxy_raw.columns if c != dxy_date_col]
        dxy_val_col = non_date[0]

df_dxy = pd.DataFrame({
    "timestamp": pd.to_datetime(df_dxy_raw[dxy_date_col], errors="coerce"),
    "dxy_close": pd.to_numeric(df_dxy_raw[dxy_val_col], errors="coerce"),
}).dropna()

df_dxy.set_index("timestamp", inplace=True)
df_dxy.sort_index(inplace=True)
if df_dxy.index.tz is not None:
    df_dxy.index = df_dxy.index.tz_localize(None)

df_dxy_hourly = df_dxy.resample("1h").ffill()

print(f"✅ DXY — {len(df_dxy):,} daily → {len(df_dxy_hourly):,} hourly rows")
del df_dxy_raw, df_dxy
gc.collect()

📖 Loading DXY Dollar Index ...
   Found: DTWEXB.csv
✅ DXY — 6,317 daily → 218,617 hourly rows


0

In [7]:
# ============================================================================
# STEP 3C: FEAR & GREED INDEX (DAILY → HOURLY)
# ============================================================================
# Scientific Alpha:
#   The Crypto Fear & Greed Index is a composite scoring retail emotion
#   (0 = Extreme Fear, 100 = Extreme Greed).  It's a proven contrarian
#   signal: extreme fear often marks local bottoms; extreme greed, tops.
# ============================================================================

print("📖 Loading Fear & Greed Index ...")
print(f"   Path: {PATH_FEAR_GREED}")

df_fg_raw = pd.read_csv(PATH_FEAR_GREED)

# Auto-detect columns
fg_date_col = _detect_ts_col(df_fg_raw.columns)

fg_val_col = None
for c in df_fg_raw.columns:
    cl = c.strip().lower()
    if "value" in cl or "greed" in cl or "fear" in cl or "fgi" in cl or "index" in cl:
        if "class" not in cl:   # skip classification columns
            fg_val_col = c
            break
if fg_val_col is None:
    fg_val_col = df_fg_raw.select_dtypes(include="number").columns[0]

df_fg = pd.DataFrame({
    "timestamp":  pd.to_datetime(df_fg_raw[fg_date_col], errors="coerce"),
    "fear_greed": pd.to_numeric(df_fg_raw[fg_val_col], errors="coerce"),
}).dropna()

df_fg.set_index("timestamp", inplace=True)
df_fg.sort_index(inplace=True)
if df_fg.index.tz is not None:
    df_fg.index = df_fg.index.tz_localize(None)

df_fg_hourly = df_fg.resample("1h").ffill()

print(f"✅ Fear & Greed — {len(df_fg):,} daily → {len(df_fg_hourly):,} hourly rows")
del df_fg_raw, df_fg
gc.collect()

📖 Loading Fear & Greed Index ...
   Path: /kaggle/input/datasets/liiucbs/crypto-fear-and-greed-index/CryptoGreedFear.csv
✅ Fear & Greed — 1,257 daily → 67,849 hourly rows


0

In [8]:
# ============================================================================
# STEP 3D: U.S. MACRO DATA — CPI & INTEREST RATES (MONTHLY → HOURLY)
# ============================================================================
# Scientific Alpha:
#   CPI (inflation) and the Federal Funds Rate define the *monetary regime*.
#   Rising rates / hot CPI → risk-off.  Falling rates / cooling CPI →
#   risk-on.  These regime shifts drive multi-month trends in BTC price.
#   By forward-filling monthly data to hourly, every row knows the
#   *current* economic backdrop without any future leakage.
# ============================================================================

print("📖 Loading U.S. Macro data ...")
print(f"   Scanning: {PATH_MACRO_DIR}\n")

# --- Robust file discovery using glob ----------------------------------------
macro_files = sorted(PATH_MACRO_DIR.glob("*.csv"))
print(f"   Found {len(macro_files)} CSV(s):")
for f in macro_files:
    print(f"     • {f.name}  ({f.stat().st_size / 1024:.0f} KB)")

# Strategy: try to find CPI and Interest Rate files by keyword matching.
# If the dataset is a single combined file, we load it and pick columns.

def _find_macro_file(files, keywords):
    """Return the first file whose name matches any keyword."""
    for f in files:
        name_lower = f.name.lower()
        if any(kw in name_lower for kw in keywords):
            return f
    return None

cpi_file  = _find_macro_file(macro_files, ["cpi", "inflation", "consumer_price", "consumer price"])
rate_file = _find_macro_file(macro_files, ["interest", "rate", "fed", "funds", "fedfunds"])

macro_dfs = []

# --- Helper to load & resample a single macro series ------------------------
def load_macro_series(filepath, value_hint, col_name):
    """Load a macro CSV, extract one numeric series, resample to hourly ffill."""
    df = pd.read_csv(filepath)
    date_col = _detect_ts_col(df.columns)

    # Find value column
    val_col = None
    for c in df.columns:
        if any(h in c.lower() for h in value_hint):
            val_col = c
            break
    if val_col is None:
        num_cols = [c for c in df.select_dtypes(include="number").columns
                    if "year" not in c.lower() and "id" not in c.lower()]
        val_col = num_cols[0] if num_cols else df.columns[-1]

    out = pd.DataFrame({
        "timestamp": pd.to_datetime(df[date_col], errors="coerce"),
        col_name:    pd.to_numeric(df[val_col], errors="coerce"),
    }).dropna()
    out.set_index("timestamp", inplace=True)
    out.sort_index(inplace=True)
    if out.index.tz is not None:
        out.index = out.index.tz_localize(None)
    return out.resample("1h").ffill()

# --- Case A: separate CPI & Rate files found --------------------------------
if cpi_file and rate_file:
    df_cpi_hourly  = load_macro_series(cpi_file,  ["cpi", "value", "index"], "cpi")
    df_rate_hourly = load_macro_series(rate_file,  ["rate", "value", "interest", "fedfunds"], "interest_rate")
    macro_dfs = [df_cpi_hourly, df_rate_hourly]
    print(f"\n   CPI  file : {cpi_file.name} → {len(df_cpi_hourly):,} hourly rows")
    print(f"   Rate file : {rate_file.name} → {len(df_rate_hourly):,} hourly rows")

# --- Case B: single combined file -------------------------------------------
elif len(macro_files) >= 1:
    combined_path = macro_files[0] if len(macro_files) == 1 else max(macro_files, key=lambda p: p.stat().st_size)
    print(f"\n   Using combined file: {combined_path.name}")
    df_macro_raw = pd.read_csv(combined_path)
    date_col = _detect_ts_col(df_macro_raw.columns)

    # Try to extract CPI column
    for c in df_macro_raw.columns:
        if "cpi" in c.lower() or "inflation" in c.lower() or "consumer" in c.lower():
            tmp = pd.DataFrame({
                "timestamp": pd.to_datetime(df_macro_raw[date_col], errors="coerce"),
                "cpi": pd.to_numeric(df_macro_raw[c], errors="coerce"),
            }).dropna().set_index("timestamp").sort_index()
            if tmp.index.tz is not None:
                tmp.index = tmp.index.tz_localize(None)
            macro_dfs.append(tmp.resample("1h").ffill())
            print(f"   → Extracted CPI from column '{c}'")
            break

    # Try to extract Interest Rate column
    for c in df_macro_raw.columns:
        if any(kw in c.lower() for kw in ["rate", "interest", "fed", "funds"]):
            tmp = pd.DataFrame({
                "timestamp": pd.to_datetime(df_macro_raw[date_col], errors="coerce"),
                "interest_rate": pd.to_numeric(df_macro_raw[c], errors="coerce"),
            }).dropna().set_index("timestamp").sort_index()
            if tmp.index.tz is not None:
                tmp.index = tmp.index.tz_localize(None)
            macro_dfs.append(tmp.resample("1h").ffill())
            print(f"   → Extracted Interest Rate from column '{c}'")
            break

    # If no keyword matched, just take the first two numeric columns
    if not macro_dfs:
        num_cols = [c for c in df_macro_raw.select_dtypes(include="number").columns
                    if "year" not in c.lower() and "id" not in c.lower()]
        for i, c in enumerate(num_cols[:2]):
            name = "cpi" if i == 0 else "interest_rate"
            tmp = pd.DataFrame({
                "timestamp": pd.to_datetime(df_macro_raw[date_col], errors="coerce"),
                name: pd.to_numeric(df_macro_raw[c], errors="coerce"),
            }).dropna().set_index("timestamp").sort_index()
            if tmp.index.tz is not None:
                tmp.index = tmp.index.tz_localize(None)
            macro_dfs.append(tmp.resample("1h").ffill())
            print(f"   → Extracted '{name}' from column '{c}'")

    del df_macro_raw
    gc.collect()
else:
    print("⚠️  No macro files found — skipping CPI/Rate columns.")

# --- Merge macro sub-dataframes into one -------------------------------------
if macro_dfs:
    df_macro_hourly = macro_dfs[0]
    for extra in macro_dfs[1:]:
        df_macro_hourly = df_macro_hourly.join(extra, how="outer")
    df_macro_hourly.ffill(inplace=True)
    df_macro_hourly.index.name = "timestamp"
    print(f"\n✅ Macro data fused — {len(df_macro_hourly):,} hourly rows, "
          f"cols: {list(df_macro_hourly.columns)}")
else:
    df_macro_hourly = pd.DataFrame()
    print("⚠️  No macro data available.")

gc.collect()

📖 Loading U.S. Macro data ...
   Scanning: /kaggle/input/datasets/eswaranmuthu/u-s-economic-vital-signs-25-years-of-macro-data

   Found 1 CSV(s):
     • macro_data_25yrs.csv  (111 KB)

   Using combined file: macro_data_25yrs.csv
   → Extracted CPI from column 'CPI'
   → Extracted Interest Rate from column 'Fed Funds Rate'

✅ Macro data fused — 63,433 hourly rows, cols: ['cpi', 'interest_rate']


0

## Step 4 — The Grand Merge

**Strategy:** Left-join everything onto `df_market` (the BTC hourly backbone). This guarantees:
1. Every row has valid OHLCV data (no phantom hours).
2. Auxiliary signals are present only where BTC was trading.
3. Sentiment NaNs → 0 (no tweets = neutral assumption). Macro NaNs → ffill (last known value persists).

In [9]:
# ============================================================================
# STEP 4: THE GRAND MERGE
# ============================================================================
# Scientific Alpha:
#   By left-joining on the market backbone we ensure temporal consistency.
#   Every row is anchored to a real BTC trading hour.  No synthetic hours,
#   no forward-looking data.  This is the foundation of a leak-free dataset.
# ============================================================================

print("🔗 Merging all signals onto the market backbone ...\n")

df_final = df_market.copy()
merge_report = {"market": len(df_final)}

# --- 1) Sentiment (hourly) --------------------------------------------------
if df_sentiment_hourly.index.tz is not None:
    df_sentiment_hourly.index = df_sentiment_hourly.index.tz_localize(None)

df_final = df_final.join(df_sentiment_hourly, how="left")
df_final["sentiment_mean"].fillna(0, inplace=True)
df_final["sentiment_std"].fillna(0, inplace=True)
df_final["tweet_volume"].fillna(0, inplace=True)
merge_report["sentiment"] = df_sentiment_hourly.index.isin(df_final.index).sum()
print(f"   ✓ Sentiment merged — matched {merge_report['sentiment']:,} hours")

# --- 2) S&P 500 (hourly ffilled) --------------------------------------------
df_final = df_final.join(df_spx_hourly, how="left")
merge_report["spx"] = df_final["spx_close"].notna().sum()
print(f"   ✓ S&P 500   merged — {merge_report['spx']:,} non-null hours")

# --- 3) DXY (hourly ffilled) ------------------------------------------------
df_final = df_final.join(df_dxy_hourly, how="left")
merge_report["dxy"] = df_final["dxy_close"].notna().sum()
print(f"   ✓ DXY       merged — {merge_report['dxy']:,} non-null hours")

# --- 4) Fear & Greed (hourly ffilled) ---------------------------------------
df_final = df_final.join(df_fg_hourly, how="left")
merge_report["fear_greed"] = df_final["fear_greed"].notna().sum()
print(f"   ✓ Fear/Greed merged — {merge_report['fear_greed']:,} non-null hours")

# --- 5) Macro (CPI + Interest Rate) -----------------------------------------
if not df_macro_hourly.empty:
    df_final = df_final.join(df_macro_hourly, how="left")
    for col in df_macro_hourly.columns:
        merge_report[col] = df_final[col].notna().sum()
        print(f"   ✓ {col:<14} merged — {merge_report[col]:,} non-null hours")

# --- Handling residual NaNs --------------------------------------------------
macro_cols = ["spx_close", "dxy_close", "fear_greed"]
if "cpi" in df_final.columns:
    macro_cols.append("cpi")
if "interest_rate" in df_final.columns:
    macro_cols.append("interest_rate")

df_final[macro_cols] = df_final[macro_cols].ffill()

# --- Anti-Data-Leakage: Macro Reporting Lag Shift ----------------------------
# CPI and Interest Rate are published with a ~15-day lag after the reporting
# period ends.  Without this shift, the model "sees" macro data 2-3 weeks
# before it was publicly available — a severe form of look-ahead bias that
# inflates backtest performance but fails catastrophically in live trading.
# We shift by 15 days × 24 hours = 360 hourly rows.
MACRO_LAG_HOURS = 15 * 24  # 15-day reporting lag in hourly resolution

if "cpi" in df_final.columns:
    df_final["cpi"] = df_final["cpi"].shift(MACRO_LAG_HOURS)
    print(f"\n   🛡️  CPI shifted by +{MACRO_LAG_HOURS} hours (15-day reporting lag)")

if "interest_rate" in df_final.columns:
    df_final["interest_rate"] = df_final["interest_rate"].shift(MACRO_LAG_HOURS)
    print(f"   🛡️  Interest Rate shifted by +{MACRO_LAG_HOURS} hours (15-day reporting lag)")

# --- Golden Window Synchronization ------------------------------------------
# Price data starts in 2018 but tweet data only begins Feb 2021.  The ~3 years
# of sentiment=0 rows carry no informational value and introduce a structural
# bias: the model learns that "zero sentiment" is the norm, diluting the
# predictive power of real sentiment signals.
#
# FIX: Using threshold > 10 instead of > 0.  Sparse noise tweets (volume 1-10)
# from bot/scraper artefacts can appear years before the real tweet dataset
# starts, causing the crop to fail silently.  A threshold of 10 ensures we
# anchor on the first hour with *meaningful* social-media coverage.
GOLDEN_THRESHOLD = 10

tweet_start_mask = df_final["tweet_volume"] > GOLDEN_THRESHOLD
if tweet_start_mask.any():
    golden_start = df_final.loc[tweet_start_mask].index.min()
    rows_before  = len(df_final)
    df_final     = df_final.loc[golden_start:].copy()
    rows_cropped = rows_before - len(df_final)
    print(f"\n   ✂️  Golden Window (threshold > {GOLDEN_THRESHOLD}):")
    print(f"       Cropped {rows_cropped:,} rows before {golden_start}")
    print(f"       Dataset now starts at: {df_final.index.min()}")
else:
    print(f"\n   ⚠️  No tweet_volume > {GOLDEN_THRESHOLD} found — skipping Golden Window crop")

# --- Explicit dropna on critical columns -------------------------------------
# FIX: The previous `df_final.dropna(inplace=True)` silently failed to remove
# rows where macro columns (cpi, interest_rate) were NaN due to the 15-day
# lag shift.  We now target the exact subset of critical columns and reassign
# explicitly to guarantee the operation takes effect.
critical_cols = ["close", "spx_close"]
if "cpi" in df_final.columns:
    critical_cols.append("cpi")
if "interest_rate" in df_final.columns:
    critical_cols.append("interest_rate")

pre_drop = len(df_final)
df_final = df_final.dropna(subset=critical_cols)
post_drop = pre_drop - len(df_final)
print(f"\n   🧹 Dropna on {critical_cols}:")
print(f"       Before: {pre_drop:,} rows")
print(f"       After:  {len(df_final):,} rows")
print(f"       Removed: {post_drop:,} rows")

# --- Dtype optimization (float64 → float32 saves ~50% RAM) ------------------
float_cols = df_final.select_dtypes(include="float64").columns
df_final[float_cols] = df_final[float_cols].astype(np.float32)

print(f"\n✅ Grand Merge complete!")
print(f"   Shape: {df_final.shape}")
print(f"   Memory: {df_final.memory_usage(deep=True).sum() / 1e6:.1f} MB")
print(f"   Date range: {df_final.index.min()} → {df_final.index.max()}")
print(f"   Columns: {list(df_final.columns)}")

# Free intermediate dataframes
del df_sentiment_hourly, df_spx_hourly, df_dxy_hourly, df_fg_hourly
if not df_macro_hourly.empty:
    del df_macro_hourly
del df_market
gc.collect()

🔗 Merging all signals onto the market backbone ...

   ✓ Sentiment merged — matched 4,037 hours
   ✓ S&P 500   merged — 67,007 non-null hours
   ✓ DXY       merged — 16,975 non-null hours
   ✓ Fear/Greed merged — 67,727 non-null hours
   ✓ cpi            merged — 63,345 non-null hours
   ✓ interest_rate  merged — 63,345 non-null hours

   🛡️  CPI shifted by +360 hours (15-day reporting lag)
   🛡️  Interest Rate shifted by +360 hours (15-day reporting lag)

   ✂️  Golden Window (threshold > 10):
       Cropped 27,046 rows before 2021-02-05 10:00:00
       Dataset now starts at: 2021-02-05 10:00:00

   🧹 Dropna on ['close', 'spx_close', 'cpi', 'interest_rate']:
       Before: 44,112 rows
       After:  44,112 rows
       Removed: 0 rows

✅ Grand Merge complete!
   Shape: (44112, 13)
   Memory: 2.6 MB
   Date range: 2021-02-05 10:00:00 → 2026-02-17 23:00:00
   Columns: ['open', 'high', 'low', 'close', 'volume', 'sentiment_mean', 'sentiment_std', 'tweet_volume', 'spx_close', 'dxy_close', '

0

## Step 5 — Validation & Export

Final quality checks before writing the master Parquet file.

In [10]:
# ============================================================================
# STEP 5: VALIDATION & EXPORT
# ============================================================================

print("=" * 70)
print("  CHIMERA 2.0 — MASTER DATASET VALIDATION REPORT")
print("=" * 70)

# --- Schema ------------------------------------------------------------------
print("\n📋 Schema (df_final.info):")
print("-" * 40)
df_final.info(memory_usage="deep")

# --- Null audit --------------------------------------------------------------
print("\n\n🔍 Null Audit:")
print("-" * 40)
null_counts = df_final.isnull().sum()
null_pct    = (df_final.isnull().mean() * 100).round(2)
null_report = pd.DataFrame({"nulls": null_counts, "pct": null_pct})
null_report = null_report[null_report["nulls"] > 0]
if null_report.empty:
    print("   ✅ ZERO nulls across all columns!")
else:
    print(null_report.to_string())
    print(f"\n   ⚠ {len(null_report)} column(s) still have nulls.")

# --- Temporal continuity check -----------------------------------------------
print("\n\n⏱️  Temporal Continuity:")
print("-" * 40)
time_diffs = df_final.index.to_series().diff().dropna()
expected   = pd.Timedelta(hours=1)
gaps       = time_diffs[time_diffs > expected]
print(f"   Expected frequency : 1H")
print(f"   Total rows         : {len(df_final):,}")
print(f"   Date range         : {df_final.index.min()} → {df_final.index.max()}")
print(f"   Gaps > 1H found   : {len(gaps):,}")
if len(gaps) > 0 and len(gaps) <= 10:
    print("   Largest gaps:")
    for ts, gap in gaps.nlargest(5).items():
        print(f"      {ts}  →  gap = {gap}")

# --- Quick statistical sanity check ------------------------------------------
print("\n\n📊 Statistical Summary (first 5 cols):")
print("-" * 40)
print(df_final.describe().iloc[:, :5].to_string())

# --- Preview -----------------------------------------------------------------
print("\n\n👀 Head:")
df_final.head(3)

  CHIMERA 2.0 — MASTER DATASET VALIDATION REPORT

📋 Schema (df_final.info):
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44112 entries, 2021-02-05 10:00:00 to 2026-02-17 23:00:00
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   open            44112 non-null  float32
 1   high            44112 non-null  float32
 2   low             44112 non-null  float32
 3   close           44112 non-null  float32
 4   volume          44112 non-null  float32
 5   sentiment_mean  44112 non-null  float32
 6   sentiment_std   44112 non-null  float32
 7   tweet_volume    44112 non-null  float32
 8   spx_close       44112 non-null  float32
 9   dxy_close       44112 non-null  float32
 10  fear_greed      44112 non-null  float32
 11  cpi             44112 non-null  float32
 12  interest_rate   44112 non-null  float32
dtypes: float32(13)
memory usage: 2.5 MB


🔍 Null Audit:
-------

,open,high,low,close,volume,sentiment_mean,sentiment_std,tweet_volume,spx_close,dxy_close,fear_greed,cpi,interest_rate
timestamp,,,,,,,,,,,,,
2021-02-05 10:00:00,37251.0703,37655.4414,37200.0000,37395.9102,2315.2617,0.3789,0.3420,11.0000,3886.8301,129.1544,66.0000,262.6390,0.0900
2021-02-05 11:00:00,37395.9219,37733.7500,37395.7812,37691.3203,2165.1494,0.1088,0.3642,88.0000,3886.8301,129.1544,66.0000,262.6390,0.0900
2021-02-05 12:00:00,37691.3203,38151.6914,37527.1406,37850.3594,4197.9575,0.2038,0.3425,139.0000,3886.8301,129.1544,66.0000,262.6390,0.0900


In [11]:
# ============================================================================
# SAVE TO PARQUET
# ============================================================================
# Parquet is columnar, compressed, and preserves dtypes — ideal for
# downstream feature engineering and ML pipelines.
# ============================================================================

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_final.to_parquet(OUTPUT_FILE, engine="pyarrow", index=True)

file_size = OUTPUT_FILE.stat().st_size / 1e6
print(f"💾 Saved: {OUTPUT_FILE}")
print(f"   Size:  {file_size:.1f} MB")
print(f"   Rows:  {len(df_final):,}")
print(f"   Cols:  {df_final.shape[1]}")

# --- Quick read-back sanity test ---------------------------------------------
df_check = pd.read_parquet(OUTPUT_FILE)
assert df_check.shape == df_final.shape, "Shape mismatch on read-back!"
assert (df_check.columns == df_final.columns).all(), "Column mismatch on read-back!"
print("\n✅ Read-back verification passed — Parquet file is healthy.")
print("\n🏁 Phase 1: Data Kitchen COMPLETE. Ready for Feature Engineering.")
del df_check
gc.collect()

💾 Saved: /kaggle/working/chimera_master_dataset.parquet
   Size:  1.8 MB
   Rows:  44,112
   Cols:  13

✅ Read-back verification passed — Parquet file is healthy.

🏁 Phase 1: Data Kitchen COMPLETE. Ready for Feature Engineering.


0